# Model Functions

### A. Softmax Classification

* Probability that $y$ is of class $i$: $ P(y = i|x,w,b) = softmax_x(w\cdot x + b) = \frac{e^{w_ix+b_i}}{\sum_je^{w_jx+b_j}}  $
* Prediction: $ yHat = argmax P(y = i|x,w,b) $

### B. Loss Function

* $ L(\theta=\{W,b\},D) = -L(\theta=\{W,b\},D) = -\sum_{i=0}^{|D|} log P(Y = y_i | x_i, W,b) $

# Model Code

In [46]:
import six.moves.cPickle as pickle
import gzip, os, sys, timeit

In [47]:
import numpy as np

In [48]:
import theano
import theano.tensor as T
from theano import function, shared
from theano.tensor.nnet import softmax

In [18]:
class LogisticRegression:
    
    def __init__(self, inpt, nIn, nOut):
        # k (1..K): class index; i (1..N): data point index; d: number of predictors.
        # W (d x K matrix): column k = weights for class k.
        # x (1 x d vector): in dataset X, row i = data point i.
        # b (1 x K vector): element k = bias for class k.
        self.W = shared(value=np.zeros((nIn,nOut),dtype=theano.config.floatX), 
                        name='W', borrow=True) # W is a matrix.
        self.b = shared(value=np.zeros((nOut,),dtype=theano.config.floatX), 
                        name='b', borrow=True) # b is a vector.
        self.pYgivenX = softmax(T.dot(inpt, self.W) + self.b)
        self.yHat = T.argmax(self.pYgivenX, axis=1)
        self.params = [self.W, self.b]
        self.inpt = inpt
        
    def nll(self, y): # negative log-likelihood.
        # type(y): T.TensorType
        return -T.mean(T.log(self.pYgivenX)[T.arange(y.shape[0]), y])
    
    def errors(self, y):
        # returns a float, the number of errors in the minibatch.
        # type(y): T.TensorType
        assert y.ndim == self.yHat.ndim
        assert y.dtype.startswith('int')
        return T.mean(T.neq(self.yHat, y)) # .neq returns a vector where 1 for a misprediction.
   

In [71]:
def shared_dataset(data, borrow=True):
    X, Y = data
    sharedX = theano.shared(np.asarray(X,dtype=theano.config.floatX), borrow=borrow)
    sharedY = theano.shared(np.asarray(Y,dtype=theano.config.floatX), borrow=borrow)
    return sharedX, T.cast(sharedY, 'int32') 
def load_mnist():
    import os
    os.chdir("/Users/jacobsw/Desktop/IMPLEMENTATION_CAMP/CODE/BASIC_TOPICS/ML_GENERAL/PYTHON_IMPL/DATA/")
    with gzip.open('mnist.pkl.gz') as f:
        data_train, data_dev, data_test = pickle.load(f)
    X_train, Y_train = shared_dataset(data_train)
    X_dev, Y_dev = shared_dataset(data_dev)
    X_test, Y_test = shared_dataset(data_test)
    
    return [(X_train, Y_train), (X_dev, Y_dev), (X_test, Y_test)] 

In [72]:
def sgd(lr=.1, epochs=1000, batchSize=600, data=load_mnist): # dataset='mnist.pkl.gz' as 3rd arg
    
    datasets = load_mnist()
    X_train, Y_train = datasets[0]
    X_dev, Y_dev = datasets[1]
    X_test, Y_test = datasets[2]

    nTrainBatches = X_train.get_value(borrow=True).shape[0] / batchSize # // if Python 3.
    nDevBatches = X_dev.get_value(borrow=True).shape[0] / batchSize
    nTestBatches = X_test.get_value(borrow=True).shape[0] / batchSize
    print "... building the model"
    index = T.iscalar() # index of a batch.
    x = T.matrix('x')
    y = T.ivector('y')
    classifier = LogisticRegression(inpt=x, nIn=28*28, nOut=10)
    cost = classifier.nll(y)
    test_model = function(inputs=[index], outputs=classifier.errors(y),
                          givens = {x: X_test[index*batchSize: (index+1)*batchSize],
                                    y: Y_test[index*batchSize: (index+1)*batchSize]})
    dev_model = function(inputs=[index], outputs=classifier.errors(y),
                         givens = {x: X_dev[index*batchSize: (index+1)*batchSize],
                                   y: Y_dev[index*batchSize: (index+1)*batchSize]})
        # if not do 'givens' trick, the batch size has to be fixed in building symbolic graph.
        # 'givens' trick modify the graph to compile before compiling it. 
        # in other words, we substitute in the graph, the key in givens with the associated value. 
    gW = T.grad(cost=cost, wrt=classifier.W)
    gb = T.grad(cost=cost, wrt=classifier.b)
    updates = [(classifier.W, classifier.W - lr*gW), (classifier.b, classifier.b - lr*gb)]
    train_model = function(inputs=[index], outputs=cost, updates=updates, 
                           givens = {x: X_train[index*batchSize: (index+1)*batchSize],
                                     y: Y_train[index*batchSize: (index+1)*batchSize]})
    print "... training the model"
    
    patience = 5000 # for early stop, but examine this many data points regardless.
    patienceIncrease = 2
    improvementThreshold = .995
    validationFrequency = min(nTrainBatches, patience/2) # examine this many batches before validation check.
    bestValidationLoss = np.inf
    testScore = .0
    startTime = timeit.default_timer()
    doneLooping = False
    epoch = 0
    while (epoch < epochs) and (not doneLooping):
        epoch += 1
        for batchIndex in range(nTrainBatches):
            avgBatchCost = train_model(batchIndex)
            iter = (epoch-1)*nTrainBatches + batchIndex
            if (iter+1) % validationFrequency == 0:
                validationLoss = [dev_model(i) for i in range(nDevBatches)]
                thisValidationLoss = np.mean(validationLoss)
                print "Epoch %i, Batch %i/%i, Validation Error %f %%" % (epoch, batchIndex+1,
                                                                         nTrainBatches, thisValidationLoss*100)
                if thisValidationLoss < bestValidationLoss:
                    if thisValidationLoss < bestValidationLoss*improvementThreshold:
                        patience = max(patience, iter*patienceIncrease)
                    bestValidationLoss = thisValidationLoss
                    testLosses = [test_model(i) for i in range(nTestBatches)]
                    testScore = np.mean(testLosses)
                    print "Epoch %i, Batch %i/%i, Test Error of Best %f %%" % (epoch, batchIndex+1,
                                                                               nTrainBatches, testScore*100)
                    with open('best_model.pkl','wb') as f:
                        pickle.dump(classifier, f)
                if patience <= iter:
                    doneLooping = True
                    break
    endTime = timeit.default_timer()
    print "Optimization complete with best validation score of %f %%, best test performance %f %% " % (bestValidationLoss*100, testScore*100)
    print "The code run for %d epochs, with %f epochs/sec" % (epoch, 1.*epoch/(endTime-startTime))
    

In [134]:
def evaluate_mnist(evaluateSize=1000, test=True):
    classifier = pickle.load(open('best_model.pkl'))
    predict_model = function(inputs=[classifier.inpt], outputs=classifier.yHat)
    datasets = load_mnist()
    if test:
        X_test, Y_test = datasets[2]
        X_test = X_test.get_value()
        correct = sum(predict_model(X_test[:evaluateSize]) == Y_test[:evaluateSize].eval())
    else:
        X_train, Y_train = datasets[0]
        X_train = X_train.get_value()
        correct = sum(predict_model(X_train[:evaluateSize]) == Y_train[:evaluateSize].eval())
    print "Accuracy: {}/{} ({}%)".format(correct,evaluateSize,float(correct)/evaluateSize*100)
    

In [74]:
sgd()

... building the model
... training the model
Epoch 1, Batch 83/83, Validation Error 13.020833 %
Epoch 1, Batch 83/83, Test Error of Best 13.052083 %
Epoch 2, Batch 83/83, Validation Error 11.572917 %
Epoch 2, Batch 83/83, Test Error of Best 11.520833 %
Epoch 3, Batch 83/83, Validation Error 10.697917 %
Epoch 3, Batch 83/83, Test Error of Best 10.635417 %
Epoch 4, Batch 83/83, Validation Error 10.239583 %
Epoch 4, Batch 83/83, Test Error of Best 10.229167 %
Epoch 5, Batch 83/83, Validation Error 9.895833 %
Epoch 5, Batch 83/83, Test Error of Best 9.906250 %
Epoch 6, Batch 83/83, Validation Error 9.572917 %
Epoch 6, Batch 83/83, Test Error of Best 9.635417 %
Epoch 7, Batch 83/83, Validation Error 9.416667 %
Epoch 7, Batch 83/83, Test Error of Best 9.364583 %
Epoch 8, Batch 83/83, Validation Error 9.281250 %
Epoch 8, Batch 83/83, Test Error of Best 9.208333 %
Epoch 9, Batch 83/83, Validation Error 9.197917 %
Epoch 9, Batch 83/83, Test Error of Best 9.020833 %
Epoch 10, Batch 83/83, Valid

In [135]:
%%time
print "Train:"
evaluate_mnist(50000, test=False)
print "Test:"
evaluate_mnist(10000)

Train:
Accuracy: 46147/50000 (92.294%)
Test:
Accuracy: 9225/10000 (92.25%)
CPU times: user 4.19 s, sys: 1.28 s, total: 5.47 s
Wall time: 5.07 s
